# 【問題1】クロスバリデーション

事前学習期間では検証データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。  
事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。

In [1]:
import pandas as pd
from sklearn.model_selection import KFold

In [2]:
df = pd.read_csv('application_train.csv')
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# 多項式特徴量を作成する
poly_features = df[['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_EMPLOYED', 'TARGET']]

# imputer for handling missing values
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns = ['TARGET'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)

from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
print('Polynomial Features shape: ', poly_features.shape)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Polynomial Features shape:  (307511, 20)


In [4]:
X = poly_features
y = poly_target

In [6]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [7]:
from sklearn.metrics import roc_auc_score

In [8]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train1, X_test1 = X[train_index], X[test_index]
    y_train1, y_test1 = y[train_index], y[test_index]
    lr.fit(X_train1, y_train1)
    y_pred1 = lr.predict(X_test1)
    scores.append(roc_auc_score(y_test1, y_pred1))
print(scores)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

[0.5288566062748563, 0.5350889784539856, 0.5310665049706016, 0.5331200313913769, 0.5339068009817595]


# 【問題2】グリッドサーチ

これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。最適なパラメータを探していくことを パラメータチューニング と呼びます。パラメータチューニングをある程度自動化する単純な方法としては グリッドサーチ があります。  
scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。  
sklearn.model_selection.GridSearchCV — scikit-learn 0.21.3 documentation  
GridSearchCVクラスには引数としてモデル、探索範囲、さらにクロスバリデーションを何分割で行うかを与えます。クロスバリデーションの機能も含まれているため、これを使用する場合はKFoldクラスを利用する必要はありません。

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [11]:
lr = LogisticRegression()
parameters = {'solver':('lbfgs', 'liblinear', 'sag', 'saga'), 'C':[0.1, 0.3, 0.5, 0.7, 0.9]}
clf = GridSearchCV(lr, parameters, n_jobs=-1)
clf.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.1, 0.3, 0.5, 0.7, 0.9],
                         'solver': ('lbfgs', 'liblinear', 'sag', 'saga')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [12]:
best_param = clf.best_params_
print(best_param)

{'C': 0.1, 'solver': 'lbfgs'}


# 【問題3】Kaggle Notebooksからの調査

KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

高い汎化性能を持つモデルを作成するためには、モデルの汎化性能を知る必要がある。  
データを学習に用いるデータと、バリデーションデータ(評価用データ)に分割する際に、
モデルを適切に評価するために、学習データとテストデータの性質を考慮してデータ分割を行う必要がある。

- hold-out法  
最も単純な方法。学習データの一部を学習に使わず、バリデーション用にとっておく。
from sklearn.model_selection import train_test_split  
欠点  
学習データとテストデータがランダムに分割されているということを前提としているため、時系列データには使用できない  
バリデーション用にとったデータが学習に使用されないため、KFold法に比べてデータを有効に使えていない。

- k-fold  
学習データを分割し、hold-out法の手続きを複数回繰り返すことで、各回のバリデーションの学習に用いるデータの量を保ちつつ、  
バリデーションの評価に用いるデータを学習データ全体とする。  
分析コンペで与えられるデータではfold数が4か5にされることが多い。  
モデルの汎化性能を評価する際、各foldにおけるスコアを平均して行うが、それぞれのfoldの目的変数と予測値を集めてデータ全体で計算する方法もある。  
RMSEでは各foldのスコアの平均はデータ全体で計算するより低くなる。


- stratified k-fold  
k-fold法で分割する際に、foldごとに含まれるクラスの割合を等しくする。  
特に多クラス分類で極端に頻度の少ないクラスがある場合に使用される。  


- group k-fold  
学習データとテストデータがランダムに分割されていない場合に使用される。  
各顧客に複数の行動履歴があり、それぞれの行動に対して予測を行うタスクの場合等。

# 【問題4】高い汎化性能のモデル作成

問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。  
その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

In [13]:
from sklearn.model_selection import StratifiedKFold

In [14]:
lr = LogisticRegression(C=0.1, solver='lbfgs')

In [15]:
skf = KFold(n_splits=5)
skf.get_n_splits(X, y)

scores2 = []
for train_index, test_index in skf.split(X, y):
    X_train1, X_test1 = X[train_index], X[test_index]
    y_train1, y_test1 = y[train_index], y[test_index]
    lr.fit(X_train1, y_train1)
    y_pred1 = lr.predict(X_test1)
    scores2.append(roc_auc_score(y_test1, y_pred1))
print(scores2)

[0.5, 0.5, 0.5, 0.5, 0.5]


ロジスティック回帰について、以下のパラメータから一番精度が高い組み合わせを、グリッドサーチを用いて探した。  
parameters = {'solver':('lbfgs', 'liblinear', 'sag', 'saga'), 'C':[0.1, 0.3, 0.5, 0.7, 0.9]}  
結果、C=0.1, solver='lbfgs'となったが、実行した結果、パラメータ指定しない場合よりも精度が悪くなった。

# 【問題5】最終的なモデルの選定

最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。  
どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。

In [16]:
parameters = {'solver':('lbfgs', 'liblinear', 'sag', 'saga'), 'C':[0.001, 0.01, 0.1, 1], 'max_iter':[10, 100, 1000]}
clf = GridSearchCV(lr, parameters, n_jobs=-1)
clf.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=0.1, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1],
                         'max_iter': [10, 100, 1000],
                         'solver': ('lbfgs', 'liblinear', 'sag', 'saga')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [17]:
best_param2 = clf.best_params_
print(best_param2)

{'C': 0.001, 'max_iter': 10, 'solver': 'lbfgs'}


In [18]:
lr = LogisticRegression(C=0.001, max_iter=10, solver='lbfgs')

In [19]:
skf = KFold(n_splits=5)
skf.get_n_splits(X, y)

scores3 = []
for train_index, test_index in skf.split(X, y):
    X_train1, X_test1 = X[train_index], X[test_index]
    y_train1, y_test1 = y[train_index], y[test_index]
    lr.fit(X_train1, y_train1)
    y_pred1 = lr.predict(X_test1)
    scores3.append(roc_auc_score(y_test1, y_pred1))
print(scores3)

[0.5, 0.5, 0.5, 0.5, 0.5]


In [21]:
df_test = pd.read_csv('application_test.csv')
drop_df_test = df_test.dropna(thresh=df_test.shape[0]*0.5, axis=1)
df_test_top3 = drop_df_dropna_test.loc[:, ['EXT_SOURCE_3', 'EXT_SOURCE_2', 'DAYS_EMPLOYED']]

df_test_top3 = df_test_top3.fillna(df_test_top3.mean())

In [23]:
poly_features2 = df_test[['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_EMPLOYED']]

imputer2 = Imputer(strategy = 'median')

# Need to impute missing values
poly_features2 = imputer.fit_transform(poly_features2)
                                  
# Create the polynomial object with specified degree
poly_transformer2 = PolynomialFeatures(degree = 3)

# Train the polynomial features
poly_transformer2.fit(poly_features2)

# Transform the features
poly_features2 = poly_transformer2.transform(poly_features2)
print('Polynomial Features shape: ', poly_features2.shape)

Polynomial Features shape:  (48744, 20)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [24]:
y_pred2 = lr.predict(poly_features2)

In [26]:
#提出用のフォーマット作成
df_submission1 = pd.concat([df_test['SK_ID_CURR'], pd.Series(y_pred2, name='TARGET')], axis=1)
df_submission1.to_csv('submission1.csv', index=False)

問2でグリッドサーチにより、一番精度がよくなるパラメータの組み合わせを探したが、  
今回、そのパラメータの項目数を増やして、一番精度がよくなるパラメータの組み合わせを求め、  
そのパラメータにより学習、推定を行なったが、結果は問2の時と変わらなかった。  

問2:  
パラメータ組み合わせ: {'solver':('lbfgs', 'liblinear', 'sag', 'saga'), 'C':[0.1, 0.3, 0.5, 0.7, 0.9]}  
一番精度が高かった組み合わせ: {'C': 0.1, 'solver': 'lbfgs'}  
問5:  
パラメータ組み合わせ: {'solver':('lbfgs', 'liblinear', 'sag', 'saga'), 'C':[0.001, 0.01, 0.1, 1], 'max_iter':[10, 100, 1000]}  
一番精度が高かった組み合わせ: {'C': 0.001, 'max_iter': 10, 'solver': 'lbfgs'}